# Classify the MNIST database using a Convolutional Neural Network

## Objectives

* create two convolutional layers
* create a function that flattens the output
* use two fully connected layers connected by ReLU
* use batch training, change the cost function accordingly

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'0.12.1'

<img align="left" src="files/old.png"> **Old Code:** Load data

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
xtr = data.train.images
ytr = data.train.labels

xte = data.test.images
yte = data.test.labels

xva = data.validation.images
yva = data.validation.labels

<img align="left" src="files/old.png"> **Old Code:** Convolutional layer

In [5]:
def convolutional_layer(input, filter_size, in_channels, out_channels, conv_strides, pool_kernel, pool_strides):

    filter_shape = [filter_size, filter_size, in_channels, out_channels]
    biases_shape = [out_channels]
    
    filter = tf.Variable(tf.truncated_normal(filter_shape))
    biases = tf.Variable(tf.truncated_normal(biases_shape))
    
    result = tf.nn.conv2d(input, filter, strides=conv_strides, padding='SAME') + biases
    result = tf.nn.max_pool(value=result,ksize=pool_kernel,strides=pool_strides,padding='SAME')
    result = tf.nn.relu(result)
    
    return result

<img align="left" src="files/new.png"> **New Code:** make a flat layer out of an image

In [6]:
def flatten_layer(input):
    
    layer_shape = input.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(input, [-1, num_features])
    
    return layer_flat, num_features

<img align="left" src="files/old.png"> **Same fully connected layer as in 1.8**

In [7]:
def fully_connected_layer(input, num_features, num_outputs):
    
    weights = tf.Variable(tf.truncated_normal([num_features, num_outputs], stddev=0.05))
    biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))

    output = tf.matmul(input, weights) + biases
    
    return output

<img align="left" src="files/modified.png"> **Modified Code:** add batch

In [8]:
def optimize(iterations, x_train, y_train, batch_size):
    
    for i in range(0,iterations):  
        x1, x2, y1, y2 = train_test_split(x_train, y_train, train_size=batch_size, random_state=i)        
        sess.run(train_step,feed_dict = {x: x1, y: y1})

<img align="left" src="files/modified.png"> **Modified Code:** add batch

In [9]:
def test_accuracy(logits,x_data,y_data):
    
    y_pred = tf.nn.softmax(logits)
    
    y_pred_cls = tf.argmax(y_pred, dimension=1)
    y_true_cls = tf.argmax(y, dimension=1)

    prediction = tf.equal(y_pred_cls, y_true_cls)    
    
    num_examples = len(x_data)
    batch_size = 8192
    all_predictions = []
    
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = x_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        all_predictions.extend(sess.run(prediction, feed_dict = {x: batch_x, y: batch_y}))    
    
    return np.mean(all_predictions)

## Build the computational graph using the defined functions

In [10]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_2d = tf.reshape(x, [-1, 28, 28, 1])

conv_1_out = convolutional_layer(x_2d, 5, 1, 32, [1, 1, 1, 1], [1, 2, 2, 1], [1, 2, 2, 1])
conv_2_out = convolutional_layer(conv_1_out, 5, 32, 64, [1, 1, 1, 1], [1, 2, 2, 1], [1, 2, 2, 1])

layer_flat, num_features = flatten_layer(conv_2_out)

layer_1 = fully_connected_layer(layer_flat, num_features, 512)
layer_1 = tf.nn.relu(layer_1)
logits = fully_connected_layer(layer_1, 512, 10)

<img align="left" src="files/modified.png"> **Modified Code:** change the cost function

In [11]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits,y)
cost = tf.reduce_mean(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

<img align="left" src="files/old.png"> **Create the session and initialize the variables**

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

<img align="left" src="files/old.png"> **Run**

In [13]:
optimize(1000, xtr, ytr, 512)

<img align="left" src="files/old.png"> **Test accuracy**

In [14]:
test_accuracy(logits,xtr,ytr)

0.9620727272727273

In [15]:
test_accuracy(logits,xte,yte)

0.95530000000000004

In [16]:
test_accuracy(logits,xva,yva)

0.95599999999999996